In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
# services
options = Options()
# options.add_argument("--headless")

driver = webdriver.Chrome(options=options)
driver.get('https://coinmarketcap.com/')

In [ ]:
table = driver.find_element(By.CLASS_NAME,'cmc-table')
table

In [ ]:
rows = table.find_elements(By.TAG_NAME,'tr')
rows

In [ ]:
# all_tds = ['', 'rank', 'name','price' .....]

for row in rows[1:11]:
    table_data = row.find_elements(By.TAG_NAME,'td')
    rank = table_data[1].text
    name = table_data[2].text
    price = table_data[3].text
    hour_change = table_data[4].text
    day_change = table_data[5].text
    seven_days_change = table_data[6].text
    market_cap = table_data[7].text
    volume = table_data[8].text
    print(rank,name,price,hour_change,day_change,seven_days_change,market_cap,volume)
    # print(table_data)

    # break

In [2]:
                                            # Complete Code
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

# Setup WebDriver
options = Options()
options.add_argument("--start-maximized")  
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(options=options)

url = 'https://coinmarketcap.com/'
driver.get(url)
time.sleep(3)  

# Slowly scroll to the bottom of the page
scroll_pause_time = 0.5 
scroll_step = 300  # pixels per scroll
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    for i in range(0, last_height, scroll_step):
        driver.execute_script(f"window.scrollTo(0, {i});")
        time.sleep(scroll_pause_time)
    
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break  
    last_height = new_height

print("✅ Finished scrolling")



soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

table = soup.find('table', {'class': 'cmc-table'})

rows = table.find_all('tr')
print(f"✅ Found {len(rows)-1} rows (excluding header)")

crypto_data = []

for row in rows[1:]: 
    cols = row.find_all('td')
    if len(cols) >= 10:
        rank = cols[1].text.strip()
        name = cols[2].text.strip()
        price = cols[3].text.strip()
        one_hour_change = cols[4].text.strip()
        twenty_four_hour_change = cols[5].text.strip()
        seven_day_change = cols[6].text.strip()
        market_cap = cols[7].text.strip()
        hr_volume = cols[8].text.strip()
        circulating_supply = cols[9].text.strip()

        crypto_data.append([
            rank, name, price, one_hour_change,
            twenty_four_hour_change, seven_day_change,
            market_cap, hr_volume, circulating_supply
        ])

for coin in crypto_data:
    print(coin)


✅ Finished scrolling
✅ Found 101 rows (excluding header)
['', 'CoinMarketCap 20 Index DTFCMC20Buy', '$200.40', '0.26%', '0.33%', '4.64%', '$6.66M$6,655,688', '$1,705,1588.50K', '33.21K CMC20']
['1', 'BitcoinBTCBuy', '$95,429.59', '0.30%', '0.01%', '5.65%', '$1.91T$1,906,485,538,480', '$33,803,004,331354.39K', '19.97M BTC']
['2', 'EthereumETHBuy', '$3,294.93', '0.30%', '0.09%', '6.98%', '$397.68B$397,681,030,837', '$22,738,806,3906.90M', '120.69M ETH']
['3', 'TetherUSDTBuy', '$0.9994', '0.01%', '0.00%', '0.09%', '$186.77B$186,768,824,726', '$76,366,656,86276.39B', '186.86B USDT']
['4', 'BNBBNBBuy', '$934.14', '0.17%', '0.58%', '4.51%', '$127.38B$127,381,222,758', '$1,866,289,1701.99M', '136.36M BNB']
['5', 'XRPXRPBuy', '$2.07', '0.48%', '0.14%', '0.54%', '$125.97B$125,974,128,276', '$2,342,303,5381.13B', '60.69B XRP']
['6', 'SolanaSOLBuy', '$144.87', '0.57%', '2.15%', '6.52%', '$82.12B$82,115,612,843', '$3,550,093,59424.50M', '565.32M SOL']
['7', 'USDCUSDCBuy', '$0.9998', '0.01%', '0.01

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from bs4 import BeautifulSoup
import time
import logging
from typing import List, Dict
from contextlib import contextmanager

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Constants
COINMARKETCAP_URL = 'https://coinmarketcap.com/'
SCROLL_PAUSE_TIME = 0.5
SCROLL_STEP = 300
MAX_SCROLL_ATTEMPTS = 5
EXPLICIT_WAIT_TIMEOUT = 10

@contextmanager
def get_chrome_driver():
    """Context manager for Chrome WebDriver to ensure proper cleanup."""
    options = Options()
    options.add_argument("--start-maximized")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    
    driver = None
    try:
        driver = webdriver.Chrome(options=options)
        driver.implicitly_wait(5)
        yield driver
    finally:
        if driver:
            driver.quit()
            logger.info("Browser closed successfully")

def scroll_to_load_content(driver: webdriver.Chrome, max_scrolls: int = MAX_SCROLL_ATTEMPTS) -> None:
    """
    Scroll the page to load dynamic content.
    
    Args:
        driver: Selenium WebDriver instance
        max_scrolls: Maximum number of scroll attempts to prevent infinite loops
    """
    scroll_attempts = 0
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while scroll_attempts < max_scrolls:
        # Smooth scroll to bottom
        for i in range(0, last_height, SCROLL_STEP):
            driver.execute_script(f"window.scrollTo(0, {i});")
            time.sleep(SCROLL_PAUSE_TIME)
        
        # Wait for new content to load
        time.sleep(1)
        
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            logger.info("Reached end of page")
            break
        
        last_height = new_height
        scroll_attempts += 1
        logger.info(f"Scroll attempt {scroll_attempts}/{max_scrolls}")
    
    logger.info("Finished scrolling")

def parse_crypto_data(soup: BeautifulSoup) -> List[Dict[str, str]]:
    """
    Parse cryptocurrency data from BeautifulSoup object.
    
    Args:
        soup: BeautifulSoup object containing the page HTML
        
    Returns:
        List of dictionaries containing cryptocurrency information
    """
    crypto_data = []
    
    table = soup.find('table', {'class': 'cmc-table'})
    if not table:
        logger.error("Could not find cryptocurrency table")
        return crypto_data
    
    rows = table.find_all('tr')
    logger.info(f"Found {len(rows)-1} cryptocurrency rows (excluding header)")
    
    for row in rows[1:]:  # Skip header row
        cols = row.find_all('td')
        if len(cols) >= 10:
            try:
                crypto_info = {
                    'rank': cols[1].text.strip(),
                    'name': cols[2].text.strip(),
                    'price': cols[3].text.strip(),
                    '1h_change': cols[4].text.strip(),
                    '24h_change': cols[5].text.strip(),
                    '7d_change': cols[6].text.strip(),
                    'market_cap': cols[7].text.strip(),
                    '24h_volume': cols[8].text.strip(),
                    'circulating_supply': cols[9].text.strip()
                }
                crypto_data.append(crypto_info)
            except (IndexError, AttributeError) as e:
                logger.warning(f"Error parsing row: {e}")
                continue
    
    return crypto_data

def scrape_coinmarketcap() -> List[Dict[str, str]]:
    """
    Main function to scrape cryptocurrency data from CoinMarketCap.
    
    Returns:
        List of dictionaries containing cryptocurrency information
    """
    try:
        with get_chrome_driver() as driver:
            logger.info(f"Navigating to {COINMARKETCAP_URL}")
            driver.get(COINMARKETCAP_URL)
            
            # Wait for table to be present
            try:
                WebDriverWait(driver, EXPLICIT_WAIT_TIMEOUT).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "cmc-table"))
                )
                logger.info("Table loaded successfully")
            except TimeoutException:
                logger.error("Timeout waiting for table to load")
                return []
            
            # Scroll to load all content
            scroll_to_load_content(driver)
            
            # Parse the page
            soup = BeautifulSoup(driver.page_source, "html.parser")
            
        # Parse crypto data
        crypto_data = parse_crypto_data(soup)
        logger.info(f"Successfully scraped {len(crypto_data)} cryptocurrencies")
        
        return crypto_data
        
    except Exception as e:
        logger.error(f"Error during scraping: {e}", exc_info=True)
        return []

# Execute the scraper
crypto_data = scrape_coinmarketcap()

# Display results
if crypto_data:
    logger.info("Displaying cryptocurrency data:")
    for coin in crypto_data[:10]:  # Display first 10 for readability
        print(f"{coin['rank']:>4} | {coin['name']:<20} | {coin['price']:<15} | "
              f"1h: {coin['1h_change']:<8} | 24h: {coin['24h_change']:<8} | "
              f"7d: {coin['7d_change']:<8}")
    
    if len(crypto_data) > 10:
        print(f"\n... and {len(crypto_data) - 10} more cryptocurrencies")
else:
    logger.warning("No data was scraped")